In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "12345"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'image.png' # image of Grazioso logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Cristian Chavez - Project 2'))),
    html.Hr(),
    html.Div(
        
#Interactive filtering options
    dcc.RadioItems(
    id='filter-type',
    options=[
        {'label': 'Water Rescue', 'value': 'W'},
        {'label': 'Mountain Rescue', 'value': 'M'},
        {'label': 'Disaster Rescue', 'value': 'D'},
        {'label': 'Reset', 'value': 'RESET'}
    ],
    value='RESET',
    labelStyle={'display':'inline-block'})
    
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
       
#Set up the features for your interactive data table to make it user-friendly for your client

        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 5,    
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('filter-type', 'value')])            
def update_dashboard(filter_type):
    
    if filter_type == 'W':
        df = pd.DataFrame(list(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                    {'$or': [
                                                        {'breed': 'Newfoundland Mix'},
                                                        {'breed': 'Labrador Retriever Mix'},
                                                        {'breed': 'Chesapeake Bay Retriever'},
                                                    ]},
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                             {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                    }]
                                            }))) 
                                           
    elif filter_type == 'M':
        df = pd.DataFrame(list(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Alaskan Malamute'},
                                                         {'breed': 'Old English Sheepdog'},
                                                         {'breed': 'Rottweiler'},
                                                         {'breed': 'Siberian Husky'}
                                                     ]},
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                             {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                     }]
                                             })))
                                            
  
    elif filter_type == 'D':
        df = pd.DataFrame(list(shelter.read_all({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'Doberman Pinscher'},
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Golden Retriever'},
                                                         {'breed': 'Bloodhound'},
                                                         {'breed': 'Rottweiler'}
                                                     ]},
                                                     {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                              {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                      }]
                                             })))
                                                     
                                            
    #Shows unfiltered data
    elif filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)


    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #pie chart
    dfp = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dfp, names='breed', title='Breeds')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app